### Connect to DuckDB and Configure S3/MinIO Access

The following cell initializes a DuckDB connection backed by a local database file and enables the `httpfs` extension to read and write data stored in MinIO via the S3 API. It sets the MinIO endpoint, disables SSL, applies path-style addressing, and loads S3 access credentials from environment variables.

In [1]:
import os
import duckdb

os.makedirs("/opt/de_project/warehouse", exist_ok=True)
db_path = "/opt/de_project/warehouse/taxi.duckdb"
con = duckdb.connect(db_path)

# Enable S3/MinIO access via DuckDB's httpfs extension
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")

# Configure DuckDB to connect to MinIO
con.execute("SET s3_endpoint = 'minio:9000';")
con.execute("SET s3_use_ssl = false;")
con.execute("SET s3_url_style = 'path';")

# Credentials
con.execute(f"SET s3_access_key_id = '{os.environ['AWS_ACCESS_KEY_ID']}';")
con.execute(f"SET s3_secret_access_key = '{os.environ['AWS_SECRET_ACCESS_KEY']}';")

con

In [2]:
row_count = con.execute("SELECT COUNT(*) FROM taxi.taxi.trips_prepared;").fetchone()[0]
row_count

2928254

### Sample 100 rows from the prepared Taxi Table

The next cell queries a subset of the prepared dataset from DuckDB, retrieves the results as a Pandas DataFrame, and displays the first few rows for inspection.

In [3]:
q = """
SELECT
    *
FROM taxi.taxi.trips_prepared
LIMIT 100;
"""
df = con.execute(q).fetchdf()
df.head()

,DOLocationID,PULocationID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,payment_type,...,avg_mph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,month,pickup_date,year
0,170,170,2,2002-12-31 22:59:39,2002-12-31 23:05:41,1,0.63,1,N,3,...,6.265193,Manhattan,Murray Hill,Yellow Zone,Manhattan,Murray Hill,Yellow Zone,01,2002-12-31,2024
1,264,137,2,2009-01-01 23:58:40,2009-01-02 00:01:40,1,0.46,1,N,2,...,9.200000,Manhattan,Kips Bay,Yellow Zone,Unknown,N/A,N/A,01,2009-01-01,2024
2,264,237,2,2009-01-01 23:30:39,2009-01-02 00:01:39,1,10.99,1,N,2,...,21.270968,Manhattan,Upper East Side South,Yellow Zone,Unknown,N/A,N/A,01,2009-01-01,2024
3,264,138,2,2009-01-01 00:24:09,2009-01-01 01:13:00,2,10.88,1,N,2,...,13.363357,Queens,LaGuardia Airport,Airports,Unknown,N/A,N/A,01,2009-01-01,2024
4,142,236,2,2023-12-31 23:56:46,2024-01-01 00:12:06,2,2.38,1,N,1,...,9.313043,Manhattan,Upper East Side North,Yellow Zone,Manhattan,Lincoln Square East,Yellow Zone,01,2023-12-31,2024


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   DOLocationID           100 non-null    int32         
 1   PULocationID           100 non-null    int32         
 2   VendorID               100 non-null    int32         
 3   tpep_pickup_datetime   100 non-null    datetime64[us]
 4   tpep_dropoff_datetime  100 non-null    datetime64[us]
 5   passenger_count        57 non-null     Int32         
 6   trip_distance          100 non-null    float64       
 7   RatecodeID             57 non-null     Int32         
 8   store_and_fwd_flag     57 non-null     object        
 9   payment_type           100 non-null    int32         
 10  fare_amount            100 non-null    float64       
 11  extra                  100 non-null    float64       
 12  mta_tax                100 non-null    float64       
 13  tip_am

### Descriptive statistics of the sampled dataset

The following cell computes summary statistics for all numeric columns in the DataFrame, allowing you to quickly inspect distributions, ranges, and potential data quality issues.


In [6]:
df.describe()

,DOLocationID,PULocationID,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,...,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_hour,dropoff_date,trip_duration_min,avg_mph,pickup_date,year
count,100.000000,100.000000,100.000000,100,100,57.0,100.000000,57.0,100.000000,100.000000,...,100.0,100.0000,57.000000,57.000000,100.000000,100,100.000000,100.000000,100,100.0
mean,159.410000,165.580000,1.530000,2023-05-05 00:02:13.920000,2023-05-05 00:19:05.920000,1.45614,2.497800,1.070175,0.770000,20.096300,...,1.0,26.8229,2.192982,0.092105,2.980000,2023-05-04 22:33:36,16.866667,8.928686,2023-05-04 20:24:00,2024.0
min,4.000000,4.000000,1.000000,2002-12-31 22:59:39,2002-12-31 23:05:41,0.0,0.000000,1.0,0.000000,3.000000,...,1.0,8.0000,0.000000,0.000000,0.000000,2002-12-31 00:00:00,0.316667,0.000000,2002-12-31 00:00:00,2024.0
25%,87.000000,122.250000,1.000000,2024-01-01 00:15:59.250000,2024-01-01 00:32:46.750000,1.0,0.400000,1.0,0.000000,10.000000,...,1.0,16.4100,2.500000,0.000000,0.000000,2024-01-01 00:00:00,7.012500,3.247443,2024-01-01 00:00:00,2024.0
50%,156.000000,158.000000,2.000000,2024-01-01 00:36:27.500000,2024-01-01 00:52:42.500000,1.0,1.470000,1.0,1.000000,15.950000,...,1.0,22.0200,2.500000,0.000000,0.000000,2024-01-01 00:00:00,13.250000,7.944843,2024-01-01 00:00:00,2024.0
75%,236.250000,236.000000,2.000000,2024-01-01 00:49:31.750000,2024-01-01 01:05:32.750000,2.0,2.970000,1.0,1.000000,28.330000,...,1.0,32.7675,2.500000,0.000000,0.000000,2024-01-01 00:00:00,22.337500,12.059567,2024-01-01 00:00:00,2024.0
max,265.000000,263.000000,2.000000,2024-01-01 00:58:16,2024-01-01 01:39:00,6.0,23.900000,5.0,4.000000,120.000000,...,1.0,127.9400,2.500000,1.750000,23.000000,2024-01-01 00:00:00,62.050000,41.285988,2024-01-01 00:00:00,2024.0
std,76.163447,66.805338,0.501614,NaN,NaN,0.867471,3.508533,0.529813,0.814701,15.515665,...,0.0,17.4750,0.827833,0.394243,7.748548,NaN,12.410828,8.067017,NaN,0.0


### Temporal Coverage Validation

Confirm whether the dataset covers the full expected day/hour range.  
This helps detect missing ingestion or partition issues.

In [8]:
con.execute("""
SELECT 
    MIN(pickup_date) AS min_date,
    MAX(pickup_date) AS max_date,
    COUNT(DISTINCT pickup_hour) AS distinct_hours
FROM taxi.taxi.trips_prepared;
""").fetchdf()

,min_date,max_date,distinct_hours
0,2002-12-31,2024-02-01,24


### Hourly aggregation

This query aggregates trips by pickup date and hour, computing trip counts and average metrics. The resulting DataFrame preview below helps confirm that the transformation and DuckDB ingestion produced coherent time-based features.

In [9]:
q = """
SELECT
    pickup_date,
    pickup_hour,
    COUNT(*) AS trips,
    AVG(trip_distance) AS avg_distance_miles,
    AVG(total_amount) AS avg_total_amount
FROM taxi.taxi.trips_prepared
GROUP BY 1, 2
ORDER BY 1, 2
LIMIT 100;
"""
df_hourly = con.execute(q).fetchdf()
df_hourly.head()

,pickup_date,pickup_hour,trips,avg_distance_miles,avg_total_amount
0,2002-12-31,22,1,0.63000,10.500000
1,2009-01-01,0,1,10.88000,68.290000
2,2009-01-01,23,2,5.72500,29.700000
3,2023-12-31,23,10,2.60100,22.462000
4,2024-01-01,0,6523,2.94086,27.438436
